# Convergence tests

## Linear Advection with Constant Coefficient

In [8]:
import math
import numpy as np
import matplotlib.pyplot as plt

#Local libraries
from basis_nodes import generate_lagrange_poly, generate_LGL_points
from SBP_matrices import first_order_D, first_order_P_Q
from RHS_1D_advection import generate_RHS_1D_advection
from SSPRK import SSPRK33

In [9]:
#Define physical domain
xlower = 0.
xupper = 2.

#Advection speed
c_advection = 1.

#Define initial conditions for convergence test
initial_c = lambda x: np.exp(-50*(x-1)**2)+1

#Define exact solution (Periodic boundary conditions)

def periodic_function(func, period, offset):
    return lambda x: func( ((x - offset) % period ) + offset )
def exact_sol(x,t):
    toeval = periodic_function(initial_c,xupper-xlower,0.)
    return toeval(x-c_advection*t)

In [10]:
#Spatial refinement
ref_level = 5
list_RHS = [[] for i in range(ref_level)]
list_xgrids = [[] for i in range(ref_level)]
list_xiLGL = [[] for i in range(ref_level)]

h_refinement = [ref_level*2**i for i in range(ref_level)]
p_refinement = [2**i for i in range(ref_level)]

for idx, h in enumerate(h_refinement):
    for p in p_refinement:
        RHS_local, xgrid_local, xiLGL_local = generate_RHS_1D_advection(xlower=xlower,
        xupper=xupper, n_elements=h,order_x=p,
        c_advection=c_advection,BC="periodic",num_flux_type="upwind")
        list_RHS[idx].append(RHS_local)
        list_xgrids[idx].append(xgrid_local)
        list_xiLGL[idx].append(xiLGL_local)

/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/basis_nodes.py:21: RuntimeWarning: invalid value encountered in divide
  return np.prod((x-x_nodes)/(xj-x_nodes), where = mask)
/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/basis_nodes.py:21: RuntimeWarning: divide by zero encountered in divide
  return np.prod((x-x_nodes)/(xj-x_nodes), where = mask)


In [16]:
#Computing errors
tfinal = 4.
CLF=0.1
error_list = [[] for i in range(ref_level)]
for i in range(ref_level):
    #h refinement
    for j in range(ref_level):
    #p refinement
        #Compute time step
        n_nodes = p_refinement[j]
        dx = (xupper-xlower)/h_refinement[i]
        dt = CLF*dx/c_advection
        print(dt)
        #Compute numerical solution
        u0 = initial_c(list_xgrids[i][j])
        u_frames, t_eval = SSPRK33(fun=list_RHS[i][j], u0=u0, dt=dt, t0=0.,tfinal=tfinal, nframes=2)
        #Compute error
        u_numerical = u_frames[-1]
        #Compute exact solution
        u_exact = exact_sol(list_xgrids[i][j],t_eval[-1]) 
        #Compute error
        error = np.linalg.norm(u_numerical-u_exact,ord=2)/np.linalg.norm(u_exact,ord=2)
        error_list[i].append(error)

0.04000000000000001
0.04000000000000001
0.04000000000000001
0.04000000000000001
0.04000000000000001
0.020000000000000004
0.020000000000000004
0.020000000000000004
0.020000000000000004
0.020000000000000004


/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/RHS_1D_advection.py:127: RuntimeWarning: overflow encountered in multiply
  return (1./len_element)*(non_split_form_interior+elem_boundary_terms)
/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/RHS_1D_advection.py:124: RuntimeWarning: invalid value encountered in matmul
  split_form_interior = -0.5*D@c_mat@u -0.5*c_mat@D@u -0.5*np.diag(u)@D@c_vec
/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/RHS_1D_advection.py:125: RuntimeWarning: invalid value encountered in matmul
  non_split_form_interior = -c_advection*D@u
/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/RHS_1D_advection.py:126: RuntimeWarning: invalid value encountered in matmul
  elem_boundary_terms = -diagPinv*(R.T@B@(f_num(u=u)-c_advection*R@u))


0.010000000000000002
0.010000000000000002
0.010000000000000002
0.010000000000000002
0.010000000000000002
0.005000000000000001
0.005000000000000001
0.005000000000000001
0.005000000000000001
0.005000000000000001
0.0025000000000000005
0.0025000000000000005
0.0025000000000000005
0.0025000000000000005
0.0025000000000000005


In [18]:
#Create a rate of convergence table with the errors
#Using pandas, but showing the rate of convergence
#instead of the error
import pandas as pd
df = pd.DataFrame(error_list)
df.columns = p_refinement
df.index = h_refinement
df



,1,2,4,8,16
5,0.0,0.0,0.0,8.313470e+10,5.066482e+102
10,0.0,0.0,0.0,1.694085e+37,inf
20,0.0,0.0,0.0,7.034621e+89,NaN
40,0.0,0.0,0.0,inf,NaN
80,0.0,0.0,0.0,NaN,NaN
